In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from src.preprocessing import prepare_data_from_ucimlrepo as prepare_data
from src.feature_selection import rf_feature_importances
from src.model_training import train_and_compare
from src.evaluation import plot_confusion_matrix, plot_feature_importances


# Komórka 2: Wczytanie danych
data_path = '../data/german_credit_data.csv'
df = pd.read_csv(data_path)
print(df.head())
print(df.info())


# Komórka 3: Eksploracja danych (EDA)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()


# Rozkład klas
sns.countplot(x='Risk', data=df)
plt.title('Rozkład klas ryzyka')
plt.show()


# Korelacje zmiennych liczbowych
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Korelacje zmiennych liczbowych')
plt.show()


# Komórka 4: Przygotowanie danych
X, y, preprocessor = prepare_data(data_path)
print('Kształt X:', X.shape)
print('Rozkład klas:', y.value_counts())


# Komórka 5: Feature importance
idx, importances = rf_feature_importances(X, y, preprocessor, top_n=20)
print('Top feature indices:', idx)


# Komórka 6: Trenowanie i porównanie modeli
results = train_and_compare(X, y, preprocessor, output_dir='../models')


# Komórka 7: Wizualizacja macierzy pomyłek dla najlepszego modelu
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['model']
X_test_trans = preprocessor.transform(X) # dla uproszczenia użyjemy całego X
disp_fig = plot_confusion_matrix(best_model, X_test_trans, y, labels=y.unique())
plt.show()


# Komórka 8: Top feature importances dla najlepszego modelu (jeśli ma attribute)
try:
fig = plot_feature_importances(best_model, feature_names=X.columns, top_n=20)
plt.show()
except:
print('Model nie posiada feature_importances_')


# Komórka 9: Zapis wyników do CSV
results_summary = [{'Model': k, 'Accuracy': v['accuracy']} for k,v in results.items()]
pd.DataFrame(results_summary).to_csv('../models/model_results.csv', index=False)